## Decision Tree 


Workflow for Decision Tree Classifier:


1. Data Preparation:

- Load the data, ensuring it's in a suitable format (e.g., list of lists).
- Identify the feature columns and the target column (class label).

2. Building the Tree:

- Recursive Function build_tree(rows, header):
  - Base Case:
    - If information gain is 0 (no further meaningful splits), create a Leaf node with majority class prediction.
  - Otherwise:
    - Find the best question to split the data using find_best_split(rows, header).
    - Partition the data into true and false branches based on the question.
    - Recursively build subtrees for both branches.
    - Construct a DecisionNode with the question, true branch, and false branch.

3. Making Predictions:

- Function `classify(row, node)`:
  - Base Case:
    - If a Leaf node is reached, return its predictions.
  - Otherwise:
    - Evaluate the Question at the current node.
    - Recursively classify the row based on the answer (true or false branch).

4. Additional Functions:

- `unique_vals(rows, col)`: Returns unique values in a column.
- `class_counts(rows, class_index=-1)`: Counts class occurrences in a dataset.
- `numeric(value)`: Checks if a value is numeric.
- `gini(rows)`: Calculates Gini impurity of a dataset.
- `info_gain(left, right, current_uncertainity)`: Calculates information gain.
- `find_best_split(rows, header)`: Finds the best question to split the data.
- `print_tree(node, spacing="")`: Visualizes the tree structure in text format.
- `print_predictions(counts)`: Prints class probabilities.

In [1]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [2]:
header = ['color', 'diameter', 'label']

In [3]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [4]:
def class_counts(rows, class_index=-1):
    counts = {}
    
    for row in rows:
        label = row[class_index]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [5]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [6]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [7]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f"Is {header[self.column]} {condition} {str(self.value)}?"

In [15]:
index = 1
match_value = 5
question = Question(index, match_value)
question.match(training_data[index])

False

In [10]:
def partition(rows, question):
    true_rows, false_rows = [], []
    
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
    

Gini Impurity = $1 - \displaystyle\sum_{i-1}^{k} p_i^{2}$

where k is the number of class labels and $p_i$ is the probability of samples belonging to the class $i$ at a given node

In [16]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / len(rows)
        impurity -= prob_of_label**2
    return impurity

Calculate **Information Gain** using Gini Impurity in the context of decision tree:


$$
\text{Information Gain} = \text{uncertainty} - \left( \frac{|S_{\text{left}}|}{|S|} \times \text{Gini}(S_{\text{left}}) + \frac{|S_{\text{right}}|}{|S|} \times \text{Gini}(S_{\text{right}}) \right)
$$


Where:
* $|S_{\text{left}}|$ and $|S_{\text{right}}|$ are the number of samples in the left and right child nodes, respectively.
* $∣S∣$ is the total number of samples at the current node.
* $\text{Gini}(S)$ is the Gini impurity of node $S$.


It calculates the information gain by subtracting the weighted impurity of two child nodes from the uncertainty(gini impurity here) of the starting node.

In [24]:
def info_gain(left, right, current_uncertainity):
    p = len(left) / (len(left) + len(right))
    return current_uncertainity - (p * gini(left) + (1-p) * gini(right))

In [25]:
## Current Uncertainity (of root node)
current_uncertainity = gini(training_data)

In [26]:
## How much information do we gain by partitioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainity)

0.1399999999999999

In [28]:
## How much information do we gain by partitioning on 'Yellow'?
true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))
info_gain(true_rows, false_rows, current_uncertainity)

0.17333333333333323

In [27]:
## How much information do we gain by partitioning on 'Red'?
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
info_gain(true_rows, false_rows, current_uncertainity)

0.37333333333333324

In [35]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14) or 'Yellow' (0.17).
# This is because, 'Red' is better at separating 'Apple' from 'Grapes' and less number of mixed-up fruits will end up in either branch.

for color in ['Red', 'Green', 'Yellow']:
    print(color)
    true_rows, false_rows = partition(training_data, Question(0, color))
    print(info_gain(true_rows, false_rows, current_uncertainity))
    print(class_counts(true_rows), class_counts(false_rows))

Red
0.37333333333333324
{'Grape': 2} {'Apple': 2, 'Lemon': 1}
Green
0.1399999999999999
{'Apple': 1} {'Apple': 1, 'Grape': 2, 'Lemon': 1}
Yellow
0.17333333333333323
{'Apple': 1, 'Lemon': 1} {'Apple': 1, 'Grape': 2}


In [60]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value and calculating the information gain."""
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows]) # Unique values in the column
        for val in values:
            question = Question(col, val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # if it doesn't split the dataset, skip it
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            # calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain > best_gain:
                best_gain, best_question = gain, question
        
    return best_gain, best_question
            

In [55]:
best_gain, best_question = find_best_split(training_data)
best_gain, best_question

(0.37333333333333324, Is diameter >= 3?)

In [41]:
class Leaf:
    def __init__(self, rows):       
        self.predictions = class_counts(rows)

In [54]:
class DecisionNode:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [56]:
def build_tree(rows):
    
    # Try partitioning the dataset on each of the unique attributes, calculate the information gain, and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain; even the best info gain is 0, we can't split any further.
    # Since we can ask no further questions, we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    
    # TODO: Negative gain? Even though find_best_split handles negative gain cases, should I check for precision errors?
    
    # If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records the best feature / value to ask at this point, as well as the branches to follow depending on the answer.
    return DecisionNode(question, true_branch, false_branch)
    

In [57]:
def print_tree(node, spacing=""):
    """Prints the decision tree in an elegant format."""
    
    # Base case: if we have reached a leaf node
    if isinstance(node, Leaf):
        print(spacing + "Predict:", node.predictions)
        return
    
    # Print the question at this node
    print(spacing + str(node.question))
    
    # Print the branches
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


In [59]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict: {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict: {'Apple': 1}
--> False:
  Predict: {'Grape': 2}


In [61]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is color == Red?
--> True:
  Predict: {'Grape': 2}
--> False:
  Is color == Yellow?
  --> True:
    Predict: {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict: {'Apple': 1}


In [62]:
training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [63]:
def classify(row, node):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    
    # Decide whether to follow the true-branch or the false-branch
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    

In [66]:
classify(training_data[0], my_tree), "True label is " + str(training_data[0][-1])

({'Apple': 1}, 'True label is Apple')

In [71]:
def print_predictions(counts):
    total = sum(counts.values())
    probs = {}
    for label in counts.keys():
        probs[label] = str(int(counts[label]) * 100 / total) + "%"
        
    return probs

In [74]:
# Confidence is lower
print_predictions(classify(training_data[1], my_tree)), "True label is " + str(training_data[1][-1])

({'Apple': '50.0%', 'Lemon': '50.0%'}, 'True label is Apple')

In [75]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [76]:
for row in testing_data:
    print("Actual: %s. Predicted: %s" % (row[-1], print_predictions(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100.0%'}
Actual: Apple. Predicted: {'Apple': '50.0%', 'Lemon': '50.0%'}
Actual: Grape. Predicted: {'Grape': '100.0%'}
Actual: Grape. Predicted: {'Grape': '100.0%'}
Actual: Lemon. Predicted: {'Apple': '50.0%', 'Lemon': '50.0%'}
